<a href="https://colab.research.google.com/github/boothmanrylan/sequencer/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and installs

In [ ]:
# standard python imports
import os

In [ ]:
# earth engine api import and authentication
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# pip installable modules

try:
    import geemap.foliumap as geemap
except:
    !pip install geemap
    import geemap.foliumap as geemap

In [ ]:
# autoreload all python modules before executing code block
%load_ext autoreload
%autoreload 2

In [ ]:
# install and import my packages from github
from google.colab import drive, auth

drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/colaboratory

# auth.authenticate_user()

# install msslib
! if [ ! -d ./msslib ]; then git clone https://github.com/boothmanrylan/msslib.git; fi
%cd msslib
! git pull
try:
    from msslib import msslib
except:
    ! pip install .
    from msslib import msslib
%cd ../

# install sequencer
! if [ ! -d ./sequencer ]; then git clone https://github.com/boothmanrylan/sequencer.git; fi
%cd sequencer
! git pull
try:
    import sequencer
except:
    ! pip install .
    import sequencer
%cd ../

# install MSSForestDisturbances
! if [ ! -d ./MSSForestDisturbances ]; then git clone https://github.com/boothmanrylan/MSSForestDisturbances.git; fi
%cd MSSForestDisturbances
! git pull
try:
    import mfd
except:
    ! pip install .
    import mfd
%cd ../

# Reload

Run these blocks if the modules have been updated since the runtime was started

In [ ]:
# reload msslib
%cd ./msslib
! git pull
%cd ../

In [ ]:
# reload sequencer
%cd ./sequencer/
! git pull
%cd ../

In [ ]:
# reload MSSForestDisturbances
%cd ./MSSForestDisturbances/
! git pull
%cd ../